In [1]:
# imports to write dynamic markdown contents
import os
from IPython.display import display, Markdown, Latex
from IPython.display import HTML

import pandas as pd
from collections import OrderedDict

import pprint
import re
import numpy as np
import matplotlib.pyplot as plt
import random

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import plotly.express as px
init_notebook_mode(connected=True)  
import glob
import uproot 
import ROOT

import platform
print('Python:', platform.python_version())

Welcome to JupyROOT 6.26/06
Python: 3.10.8


In [2]:
# run = 10645
# run = 10647
# run = 10648
#  filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_pedestal-*{run}-0000.prdf.root")

# run = 10661
# filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_pedestal-*{run}-0000.prdf_1.root")

# run = 10737
# run = 10915
# filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_beam-*{run}-0000.prdf.root")

# run = 10762
# run = 10771
# filenames = glob.glob(f"/sphenix/u/adityadash/sPHENIX_github/analysis/TPC/DAQ/macros/beam-datalocal/TPC_ebdc*_beam-*{run}-0000.prdf_TPCRawDataTree.root")

run = 11073
filenames = glob.glob(f"/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc*_calib-*{run}-0000.prdf.root")

filenames.sort()
pprint.pprint(filenames)

['/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc00_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc01_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc03_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc04_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc05_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc06_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc07_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc08_calib-00011073-0000.prdf.root',
 '/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc09_calib-00011073-0000.pr

In [3]:
files = dict()
for filename in filenames: 
    file = ROOT.TFile.Open(filename)
    sector = re.match(".*ebdc([0-9][0-9])_.*", filename).group(1)
    files[sector] = file


pprint.pprint(files);

{'00': <cppyy.gbl.TFile object at 0x93e6710>,
 '01': <cppyy.gbl.TFile object at 0x96775c0>,
 '02': <cppyy.gbl.TFile object at 0x9615840>,
 '03': <cppyy.gbl.TFile object at 0x96f6b60>,
 '04': <cppyy.gbl.TFile object at 0x9669870>,
 '05': <cppyy.gbl.TFile object at 0x96eed70>,
 '06': <cppyy.gbl.TFile object at 0x9606550>,
 '07': <cppyy.gbl.TFile object at 0x96090e0>,
 '08': <cppyy.gbl.TFile object at 0x9716410>,
 '09': <cppyy.gbl.TFile object at 0x9718e30>,
 '10': <cppyy.gbl.TFile object at 0x9726a10>,
 '11': <cppyy.gbl.TFile object at 0x9729a50>,
 '12': <cppyy.gbl.TFile object at 0x972c510>,
 '13': <cppyy.gbl.TFile object at 0x9772c40>,
 '14': <cppyy.gbl.TFile object at 0x97757e0>,
 '15': <cppyy.gbl.TFile object at 0x9778310>,
 '16': <cppyy.gbl.TFile object at 0x978b2d0>,
 '17': <cppyy.gbl.TFile object at 0x978e050>,
 '18': <cppyy.gbl.TFile object at 0x9790c10>,
 '19': <cppyy.gbl.TFile object at 0x9793600>,
 '20': <cppyy.gbl.TFile object at 0x9796300>,
 '21': <cppyy.gbl.TFile object at 

In [4]:
files['02'].ls()
files['02'].PacketTree.Show(0)
files['02'].SampleTree.Show(0)
files['02'].TaggerTree.Show(0)

TFile**		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_calib-00011073-0000.prdf.root	
 TFile*		/phenix/u/jinhuang/links/sPHENIX_work/TPC/commissioning/TPC_ebdc02_calib-00011073-0000.prdf.root	
  KEY: TTree	PacketTree;1	Each entry is one packet
  KEY: TTree	SampleTree;1	Each entry is one waveform
  KEY: TTree	TaggerTree;1	Each entry is one tagger for level 1 trigger or endat tag
======> EVENT:0
 packet          = 4020
 frame           = 2
 nWaveormInFrame = 1036
 nTaggerInFrame  = 2
 maxFEECount     = 26
======> EVENT:0
 packet          = 4020
 frame           = 2
 nWaveormInFrame = 1036
 maxFEECount     = 26
 nSamples        = 360
 adcSamples      = 69, 
                  73, 69, 66, 70, 71, 72, 66, 66, 70, 69, 
                  73, 72, 72, 70, 68, 72, 70, 71, 68
 fee             = 22
 sampaAddress    = 2
 sampaChannel    = 26
 Channel         = 90
 BCO             = 845736
 checksum        = 37461
 checksumError   = 0
======> EVENT:0
 packet          = 4020
 fram

In [5]:
%%cpp -d

#include<sPhenixStyle.C>

In [6]:
%%cpp

SetsPhenixStyle()

sPhenixStyle: Applying nominal settings.
sPhenixStyle: ROOT6 mode


In [7]:
%jsroot on

In [8]:
c1 = ROOT.TCanvas( 'checksumErrorQA', 'checksumErrorQA', 1000, 1200)

c1.Divide(4,6)

i = 1
for sector, file in files.items():
    
    c1.cd(i)
    i=i+1
    
    hAllWaveForm = ROOT.TH1F(f'hAllWaveForm{sector}', ';fee*8+sampaAddress [0..207]', 26*8, -.5, 26*8 -.5);
    hErrorWaveForm = ROOT.TH1F(f'hErrorWaveForm{sector}', f';Sector{sector}: fee*8+sampaAddress [0..207];Probability of CheckSum Error', 26*8, -.5, 26*8 -.5);

    try:
        files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hAllWaveForm{sector}","","goff");
        files[sector].SampleTree.Draw(f"fee*8+sampaAddress >> hErrorWaveForm{sector}","checksumError>0","goff");

        hErrorWaveForm.Divide (hErrorWaveForm, hAllWaveForm, 1, 1, 'B');
        hErrorWaveForm.DrawClone();
    except: 
        pass
c1.Draw()

In [9]:

c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1000, 1200)

c1.Divide(4,6)

i = 1
for sector, file in files.items():
    

    pad = c1.cd(i)
    i=i+1

    pad.SetLogz()
    
    try:
        hADC = ROOT.TH2F(f'hADC{sector}', f';Sector{sector}: ADC Time bin [1/20MHz]; ADC [ADU]', 360, 0, 360, 256,0,1024);

        files[sector].SampleTree.Draw(f"adcSamples:Iteration$>>hADC{sector}","checksumError==0","colz");

        hADC.DrawClone("colz");
    except: 
        pass
    
c1.Draw()

In [10]:

c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1000, 1200)

c1.Divide(4,6)

i = 1
for sector, file in files.items():
    

    pad = c1.cd(i)
    i=i+1

    pad.SetLogz()
    pad.SetLogy()
    
    try:
        hADC = ROOT.TH2F(f'hADCPedstal{sector}', f';Sector{sector}: ADC Time bin [1/20MHz]; ADC [ADU]', 360, 0, 360, 255,4,1024);

        files[sector].SampleTree.Draw(f"adcSamples - adcSamples[0]:Iteration$>>hADCPedstal{sector}","checksumError==0","colz");

        hADC.DrawClone("colz");
    except: 
        pass
    
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ADCQA


In [11]:
c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1000, 400)

i = 1
sector='06'
    
ROOT.gPad.SetLogz()

hADC = ROOT.TH2F(f'hADCZ{sector}', f';Sector{sector}: ADC Time bin [1/20MHz]; ADC [ADU]', 360, 0, 360, 256,0,1024);
files[sector].SampleTree.Draw(f"adcSamples:Iteration$>>hADCZ{sector}","checksumError==0","colz");

hADC.DrawClone("colz");

    
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ADCQA


In [16]:
c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1000, 400)

sector='06'
    
ROOT.gPad.SetLogz()
ROOT.gPad.SetLogy()

hADC = ROOT.TH2F(f'hADCZZ{sector}', f';Sector{sector}: ADC Time bin [1/20MHz]; ADC [ADU]', 360, 0, 360, 256, 0, 1024);
files[sector].SampleTree.Draw(f"adcSamples - adcSamples[0]:Iteration$>>hADCZZ{sector}","checksumError==0","colz");

hADC.DrawClone("colz");

    
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ADCQA
Warning in <TFile::Append>: Replacing existing TH1: hADCZZ06 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADCZZ06 (Potential memory leak).


In [14]:
c1 = ROOT.TCanvas( 'ADCQA', 'ADCQA', 1000, 400)

sector='06'
    
ROOT.gPad.SetLogz()
ROOT.gPad.SetLogy()

hADC = ROOT.TH2F(f'hADCZZFEE{sector}', f';Sector{sector}: FEE ID [0...25]; ADC [ADU]', 26, -.5, 25.5, 256, 0, 1024);
files[sector].SampleTree.Draw(f"adcSamples - adcSamples[0]:fee>>hADCZZFEE{sector}","checksumError==0","colz");

hADC.DrawClone("colz");

    
c1.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: ADCQA
Warning in <TFile::Append>: Replacing existing TH1: hADCZZFEE06 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: hADCZZFEE06 (Potential memory leak).
